In [1]:
import numpy as np
import pandas as pd
!pip install gensim
from sklearn import utils #for shuffling of data

In [2]:
df = pd.read_csv('uci-news-aggregator.csv') # loading our data set
df.head(3)

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550


In [4]:
a=df['TITLE'][1]
print(a)

Fed's Charles Plosser sees high bar for change in pace of tapering


In [5]:
df.columns

Index(['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME',
       'TIMESTAMP'],
      dtype='object')

In [6]:
df['PUBLISHER'].value_counts() #frequency of each Publisher in the datset

Reuters                            3902
Huffington Post                    2455
Businessweek                       2395
Contactmusic.com                   2334
Daily Mail                         2254
                                   ... 
Broadway Blog \(blog\)                1
Breckenridge American                 1
SanFranciscoSentinel.com              1
Wicked Local North Attleborough       1
Journeyman Weekly                     1
Name: PUBLISHER, Length: 10985, dtype: int64

In [7]:
news=df[['TITLE','PUBLISHER']] #news datset containing only article text(TITLE) and publisher name
print(news.head())

                                               TITLE          PUBLISHER
0  Fed official says weak data caused by weather,...  Los Angeles Times
1  Fed's Charles Plosser sees high bar for change...           Livemint
2  US open: Stocks fall after Fed official hints ...       IFA Magazine
3  Fed risks falling 'behind the curve', Charles ...       IFA Magazine
4  Fed's Plosser: Nasty Weather Has Curbed Job Gr...          Moneynews


In [8]:
#filtering articles of more frequent publishers
x=(news['PUBLISHER'].value_counts()>2000 )& (news['PUBLISHER'].value_counts()<3000)

In [9]:
x[x]

Huffington Post     True
Businessweek        True
Contactmusic.com    True
Daily Mail          True
NASDAQ              True
Examiner.com        True
Name: PUBLISHER, dtype: bool

In [10]:
required_publishers=x[x].index
news_articles=news[news['PUBLISHER'].isin(required_publishers)]

In [11]:
news_articles.sample(20)

,TITLE,PUBLISHER
175712,The cast of 'Star Wars: Episode VII' is revealed,Examiner.com
168212,Miley Cyrus - Miley Cyrus is 'much better' aft...,Contactmusic.com
266794,Amazon vs. Hachette: A Custody Battle for the ...,Huffington Post
269907,Proving she's one of the glitterati! Naomi Cam...,Daily Mail
280045,Christie Dancing With Fallon Jokes He Could Be...,Businessweek
292024,'Game Of Thrones' Season 4 Episode 10 Recap: '...,Huffington Post
408125,Syfy Master Plan for Sharknado 2 Actually Worked,Businessweek
130887,Miley Cyrus - Miley Cyrus Facing Potential Mon...,Contactmusic.com
290033,US Stock-Index Futures Drop as US Weighs Iraq ...,Businessweek
284671,"Dragons Are ""Real To Us,"" Says 'How To Train Y...",Contactmusic.com


In [12]:
news_articles['PUBLISHER'].value_counts()

Huffington Post     2455
Businessweek        2395
Contactmusic.com    2334
Daily Mail          2254
NASDAQ              2228
Examiner.com        2085
Name: PUBLISHER, dtype: int64

In [13]:
news_articles

,TITLE,PUBLISHER
5,Plosser: Fed May Have to Accelerate Tapering Pace,NASDAQ
15,Forex - Pound drops to one-month lows against ...,NASDAQ
19,"Euro Anxieties Wane as Bunds Top Treasuries, S...",Businessweek
20,Noyer Says Strong Euro Creates Unwarranted Eco...,Businessweek
63,EBay CEO Donahoe's Pay Drops by 53% to $13.8 M...,Businessweek
...,...,...
422198,Overweight kids are bullying targets,Examiner.com
422277,"Deadly Ebola could affect up to 20000 people, ...",Daily Mail
422300,UN: Ebola Could Eventually Infect 20000 People,Huffington Post
422309,"Ebola Cases May Surpass 20000, WHO Says in Upd...",Businessweek


In [14]:
news_articles['PUBLISHER']

5                  NASDAQ
15                 NASDAQ
19           Businessweek
20           Businessweek
63           Businessweek
               ...       
422198       Examiner.com
422277         Daily Mail
422300    Huffington Post
422309       Businessweek
422319         Daily Mail
Name: PUBLISHER, Length: 13751, dtype: object

In [15]:
def label_text(corpus, publisher_name) :
    doc = []
    # for i, text in enumerate(corpus) :
    text =enumerate(corpus)
        # label = publisher_name + '_' + str(i)
    label = publisher_name
    doc.append(TaggedDocument(text.split(), [label]))
    return doc

In [16]:
news_articles['TITLE'].iloc[0]

'Plosser: Fed May Have to Accelerate Tapering Pace'

In [17]:
for index, row in news_articles.iterrows():
    # print(row)
    row["TITLE"]=  row["TITLE"].lower().split()

In [18]:
news_articles.head()

,TITLE,PUBLISHER
5,"[plosser:, fed, may, have, to, accelerate, tap...",NASDAQ
15,"[forex, -, pound, drops, to, one-month, lows, ...",NASDAQ
19,"[euro, anxieties, wane, as, bunds, top, treasu...",Businessweek
20,"[noyer, says, strong, euro, creates, unwarrant...",Businessweek
63,"[ebay, ceo, donahoe's, pay, drops, by, 53%, to...",Businessweek


In [20]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.model_selection import train_test_split

In [21]:
documents = []
for index, row in news_articles.iterrows():
    # print(row)
    # row["TITLE"]=  row["TITLE"].lower().split()
    label = row["PUBLISHER"]
    documents.append(TaggedDocument(row["TITLE"], [label]))

In [22]:
documents

[TaggedDocument(words=['plosser:', 'fed', 'may', 'have', 'to', 'accelerate', 'tapering', 'pace'], tags=['NASDAQ']),
 TaggedDocument(words=['forex', '-', 'pound', 'drops', 'to', 'one-month', 'lows', 'against', 'euro'], tags=['NASDAQ']),
 TaggedDocument(words=['euro', 'anxieties', 'wane', 'as', 'bunds', 'top', 'treasuries,', 'spain', 'debt', 'rallies'], tags=['Businessweek']),
 TaggedDocument(words=['noyer', 'says', 'strong', 'euro', 'creates', 'unwarranted', 'economic', 'pressure', '(1)'], tags=['Businessweek']),
 TaggedDocument(words=['ebay', 'ceo', "donahoe's", 'pay', 'drops', 'by', '53%', 'to', '$13.8', 'million', 'for', '2013'], tags=['Businessweek']),
 TaggedDocument(words=["mcdonald's", 'blames', 'weak', 'sales', 'on', 'weather'], tags=['Huffington Post']),
 TaggedDocument(words=['stock', 'market', 'news', 'for', 'march', '10,', '2014', '-', 'market', 'news'], tags=['NASDAQ']),
 TaggedDocument(words=['u.s.', 'stocks', 'open', 'lower', 'in', 'subdued', 'trade;', 'dow', 'jones', 'do

In [23]:
len(documents)

13751

In [24]:
from sklearn import utils

model_dbow = Doc2Vec(dm=0, vector_size=3000, negative=5, min_count=1, alpha=0.09, min_alpha=0.09)
model_dbow.build_vocab(documents)

for epoch in range(30):
    model_dbow.train(utils.shuffle(documents), total_examples=len(documents), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

In [35]:
test_news='forex - pound drops to one-month lows against euro'
#original source of test_news - NASDAQ
newvec = model_dbow.infer_vector(test_news.split())
predicted_publisher = model_dbow.docvecs.most_similar([newvec],topn=5)

C:\Users\HP\AppData\Local\Temp/ipykernel_17900/690109941.py:4: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  predicted_publisher = model_dbow.docvecs.most_similar([newvec],topn=5)


In [36]:
predicted_publisher

[('NASDAQ', 0.8348314166069031),
 ('Businessweek', 0.7831403613090515),
 ('Contactmusic.com', 0.5443477630615234),
 ('Examiner.com', 0.4826135039329529),
 ('Huffington Post', 0.44007033109664917)]

In [27]:
NASDAQ_vector= model_dbow.docvecs['NASDAQ']
print(NASDAQ_vector)

[-0.05797603 -0.04706192 -0.0735896  ... -0.06206594 -0.00598953
  0.0523116 ]


C:\Users\HP\AppData\Local\Temp/ipykernel_17900/4176355465.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  NASDAQ_vector= model_dbow.docvecs['NASDAQ']


In [28]:
type(newvec)

numpy.ndarray

In [29]:
# compute cosine similarity
from scipy import spatial

result = 1 - spatial.distance.cosine(newvec, NASDAQ_vector)

In [30]:
print(result)

0.8320320248603821


In [31]:
# confirming compute cosine similarity by other method
import numpy as np
from numpy.linalg import norm
cosine = np.dot(newvec,NASDAQ_vector)/(norm(newvec)*norm(NASDAQ_vector))
print("Cosine Similarity:", cosine)

Cosine Similarity: 0.832032


In [32]:
Businessweek_vector= model_dbow.docvecs['Businessweek']
print(Businessweek_vector)

[-0.03517766 -0.01668465 -0.02846768 ...  0.12602231  0.04605604
  0.08319694]


C:\Users\HP\AppData\Local\Temp/ipykernel_17900/775424132.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  Businessweek_vector= model_dbow.docvecs['Businessweek']


In [33]:
result = 1 - spatial.distance.cosine(newvec, Businessweek_vector)
print(result)

0.7880657911300659


In [34]:
Huffington_Post_vector= model_dbow.docvecs['Huffington Post']
print(Businessweek_vector)
result = 1 - spatial.distance.cosine(newvec, Huffington_Post_vector)
print(result)

[-0.03517766 -0.01668465 -0.02846768 ...  0.12602231  0.04605604
  0.08319694]
0.4492065906524658


C:\Users\HP\AppData\Local\Temp/ipykernel_17900/4163233747.py:1: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  Huffington_Post_vector= model_dbow.docvecs['Huffington Post']
